In [1]:
! pip install torch torchvision

In [2]:
import torch, os, sys
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models

root_dir = os.path.abspath(os.path.join('..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)
from src.dataset import RealCartoon

In [3]:
DATASET_DIR = f"..{os.sep}dataset{os.sep}IIIT-CFW1.0"
CARTOON_DATASET = "cartoonFaces"
REAL_FACES = "realFaces"
DATASET_FILE = "dataset.csv"

RANDOM_STATE = 1
EPOCHS = 5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# Define the data transformations
transform = transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomHorizontalFlip(p=0.25),
            transforms.RandomVerticalFlip(p=0.25),
            transforms.ColorJitter(brightness=(0.8,1), contrast=(0.8,1)),
            transforms.RandomPerspective(),
            transforms.RandomRotation(degrees=(0, 60)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the dataset
dataset = torchvision.datasets.ImageFolder('../dataset/IIIT-CFW1.0/', transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [5]:
# Load the pre-trained ResNet-18 model
model = models.shufflenet_v2_x1_0(pretrained=True)

# Freeze the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your custom dataset
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

print(f"Training on {device}")

for epoch in range(EPOCHS):
    # Training loop
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = correct / total

    print(f'Epoch [{epoch + 1}/{EPOCHS}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')


# Define the path to save the model
model_path = '../models/shufflenet_v2_x1_0_model.pth'

# Save the model
torch.save(model.state_dict(), model_path)

/home/mutakabbir/Projects/real-cartoon/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/mutakabbir/Projects/real-cartoon/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on cuda
Epoch [1/5], Loss: 0.4734, Accuracy: 90.03%
Epoch [2/5], Loss: 0.3428, Accuracy: 90.03%
Epoch [3/5], Loss: 0.3269, Accuracy: 90.03%
Epoch [4/5], Loss: 0.3239, Accuracy: 90.03%
Epoch [5/5], Loss: 0.3188, Accuracy: 90.03%


In [6]:
# Load the pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)

# Freeze the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your custom dataset
num_classes = len(dataset.classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

print(f"Training on {device}")

for epoch in range(EPOCHS):
    # Training loop
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = correct / total

    print(f'Epoch [{epoch + 1}/{EPOCHS}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

# Define the path to save the model
model_path = '../models/resnet18_model.pth'

# Save the model
torch.save(model.state_dict(), model_path)


/home/mutakabbir/Projects/real-cartoon/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training on cuda
Epoch [1/5], Loss: 0.2113, Accuracy: 95.22%
Epoch [2/5], Loss: 0.1330, Accuracy: 95.12%
Epoch [3/5], Loss: 0.1278, Accuracy: 95.87%
Epoch [4/5], Loss: 0.1231, Accuracy: 95.52%
Epoch [5/5], Loss: 0.1158, Accuracy: 95.47%


In [8]:
# # Load the pre-trained ResNet-18 model
# model = models.efficientnet_b7(progress=True, pretrained=True)

# # Freeze the pre-trained layers
# for param in model.parameters():
#     param.requires_grad = False

# # Modify the last fully connected layer to match the number of classes in your custom dataset
# num_classes = len(dataset.classes)
# model.fc = nn.Linear(model.fc.in_features, num_classes)
# model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.fc.parameters(), lr=0.001, momentum=0.9)

# print(f"Training on {device}")

# for epoch in range(EPOCHS):
#     # Training loop
#     model.train()
#     running_loss = 0.0
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     # Validation loop
#     model.eval()
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for images, labels in val_loader:
#             images, labels = images.to(device), labels.to(device)

#             outputs = model(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     epoch_loss = running_loss / len(train_loader)
#     accuracy = correct / total

#     print(f'Epoch [{epoch + 1}/{EPOCHS}], Loss: {epoch_loss:.4f}, Accuracy: {accuracy * 100:.2f}%')

# # Define the path to save the model
# model_path = '../models/resnet18_model.pth'

# # Save the model
# torch.save(model.state_dict(), model_path)
